<a href="https://colab.research.google.com/github/JTT94/nlp_neural_process/blob/master/Class_BERT_NeuralProcesses_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re

from keras import backend as K
import numpy as np
import string
from datetime import datetime 
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd

Using TensorFlow backend.


In [0]:
tf.logging.set_verbosity(tf.logging.INFO)

In [0]:
!pip install bert-tensorflow
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization


     |████████████████████████████████| 71kB 1.4MB/s 


W0808 11:04:44.523133 139698332669824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# output_directory = "./gdrive/My Drive/Kaggle_toxic_comments/test_output"
# output_directory = "./test_output"

In [0]:
from google.colab import auth
auth.authenticate_user()

DATAFLAG = 'unhealthy'
BUCKET = 'rail_jigsaw_modeling' #@param {type:"string"}
MODEL = 'batch32' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/{}/models/{}'.format(BUCKET, DATAFLAG, MODEL)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: gs://rail_jigsaw_modeling/unhealthy/models/batch32 *****


In [0]:
# initialiase tensorboard 
# from https://stackoverflow.com/questions/47818822/can-i-use-tensorboard-with-google-colab


# Get TensorBoard running in the background. 
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(OUTPUT_DIR)
)

In [0]:
# # #Download and unzip ngrok. 
!test -e ngrok-stable-linux-amd64.zip || wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!test -e ngrok || unzip ngrok-stable-linux-amd64.zip


--2019-08-08 11:05:39--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.195.49.195, 52.22.236.254, 52.204.136.9, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.195.49.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13607069 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  12.98M  37.6MB/s    in 0.3s    

2019-08-08 11:05:39 (37.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13607069/13607069]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
# #Launch ngrok background process...
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://7708afdf.ngrok.io


In [0]:
import os, sys
sys.path.append('../') # add personal code dir to path for import


!test -d neural_process || git clone https://github.com/JTT94/nlp_neural_process.git neural_process
sys.path.append('./neural_process/')

import random
from neural_process import split_context_target, NeuralProcessParams
from neural_process.network import *
from neural_process.loss import *
from neural_process.predict import *
from neural_process.process import *

from neural_process.tf_model_builder_AUC import *
from neural_process.bert_utils import *

Cloning into 'neural_process'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 152 (delta 93), reused 132 (delta 79), pack-reused 0
Receiving objects: 100% (152/152), 794.27 KiB | 5.29 MiB/s, done.
Resolving deltas: 100% (93/93), done.


In [0]:
# Initialize session
sess = tf.Session()
K.set_session(sess)
K.tensorflow_backend._get_available_gpus()


W0808 11:05:45.509979 139698332669824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0808 11:05:45.511689 139698332669824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
# # filename = './cleaned_data.csv'
# filename = './gdrive/My Drive/Oxford/RAIL/Jigsaw/Technical Team/toxic_comments/Data/data1.csv'
# df = pd.read_csv(filename, index_col=0)
# cols = ['comment','antagonize', 'condescending', 'dismissive', 'generalisation', 'hostile', 'sarcastic']
# # cols = ['comment','cleaned_comment','antagonize', 'condescending', 'dismissive', 'generalisation', 'hostile', 'sarcastic', 'healthy']
# df = df[cols]

# score_column = ['antagonize', 'condescending', 'dismissive', 'generalisation', 'hostile', 'sarcastic']
# # text_col_name = 'cleaned_comment'
# text_col_name = 'comment'

# df_train = df.sample(frac = 0.8)
# df_test = df[~df.isin(df_train)].dropna()

# df_train[score_column] = df_train[score_column].round()
# df_test[score_column] = df_test[score_column].round()
# #--------

# # ### For Unhealthy dataset

cols = ['comment','antagonize', 'condescending', 'dismissive', 'generalisation', 'hostile', 'sarcastic', '_trusted_judgments']
score_column = ['antagonize', 'condescending', 'dismissive', 'generalisation', 'hostile', 'sarcastic']
text_col_name = 'comment'

## Training data

train_filename = './gdrive/My Drive/unhealthy_rail/unhealthy_train.csv'

df_train = pd.read_csv(train_filename)
df_train = df_train[cols].round()

#Cast to float - because scores and labels need to be concattenated in the model function, and so need to be same type
for i in score_column:
  df_train[i] = pd.to_numeric(df_train[i],downcast='float')

  
## Validation data

val_filename = './gdrive/My Drive/unhealthy_rail/unhealthy_val.csv'

df_val = pd.read_csv(val_filename)
df_val = df_val[cols].round()

#Cast to float - because scores and labels need to be concattenated in the model function, and so need to be same type
for i in score_column:
  df_val[i] = pd.to_numeric(df_val[i],downcast='float')

## Test data

test_filename = './gdrive/My Drive/unhealthy_rail/unhealthy_test.csv'

df_test = pd.read_csv(test_filename)
df_test = df_test[cols].round()

#Cast to float - because scores and labels need to be concattenated in the model function, and so need to be same type
for i in score_column:
  df_test[i] = pd.to_numeric(df_test[i],downcast='float')



# # # ### For kaggle dataset

# ## Training data

# # filename = './gdrive/My Drive/Data/train.csv'
# train_filename = './gdrive/My Drive/Kaggle_toxic_comments/finalSplit/train_kaggle.csv'

# df = pd.read_csv(train_filename)
# cols = ['comment_text','toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
# df = df[cols]

# score_column = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# text_col_name = 'comment_text'

# #Cast to float - because scores and labels need to be concattenated in the model function, and so need to be same type
# for i in score_column:
#   df[i] = pd.to_numeric(df[i],downcast='float')

# df_train = df[df.comment_text.str.len() <= 250]

# ## Validation data

# # filename = './gdrive/My Drive/Data/train.csv'
# val_filename = './gdrive/My Drive/Kaggle_toxic_comments/finalSplit/val_kaggle.csv'

# df = pd.read_csv(val_filename)
# cols = ['comment_text','toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
# df = df[cols]

# score_column = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# text_col_name = 'comment_text'

# #Cast to float - because scores and labels need to be concattenated in the model function, and so need to be same type
# for i in score_column:
#   df[i] = pd.to_numeric(df[i],downcast='float')

# df_val = df[df.comment_text.str.len() <= 250]

# ## Test data

# # filename = './gdrive/My Drive/Data/train.csv'
# test_filename = './gdrive/My Drive/Kaggle_toxic_comments/finalSplit/test_kaggle.csv'

# df = pd.read_csv(test_filename)
# cols = ['comment_text','toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
# df = df[cols]

# score_column = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# text_col_name = 'comment_text'

# #Cast to float - because scores and labels need to be concattenated in the model function, and so need to be same type
# for i in score_column:
#   df[i] = pd.to_numeric(df[i],downcast='float')

# df_test = df[df.comment_text.str.len() <= 250]

In [0]:
# data = pd.concat([df_train,test_df])

# data =  data.sample(frac = 1.0)

# msk = np.random.rand(len(data)) < 0.8

# train = data[msk]

# temp = data[~msk]

# msk2 = np.random.rand(len(temp)) <= 0.5

# val = temp[msk2]

# test = temp[~msk2]

In [0]:
# train.to_csv("train_kaggle.csv")
# test.to_csv("test_kaggle.csv")
# val.to_csv("val_kaggle.csv")

In [0]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_model_hub = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
tokenizer = create_tokenizer_from_hub_module(BERT_model_hub)

I0808 11:05:55.061700 139698332669824 saver.py:1499] Saver not created because there are no variables in the graph to restore
W0808 11:05:55.204028 139698332669824 deprecation_wrapper.py:119] From ./neural_process/neural_process/bert_utils.py:39: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0808 11:05:57.029646 139698332669824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [0]:
# #Pre process data for bert embedding
max_seq_length = 128

import pickle

# train_input_examples = create_examples(df_train.sample(100), score_column, text_col_name)
# # # test_input_examples = create_examples(df_test, score_column, text_col_name)

# train_features = convert_examples_to_features(train_input_examples, max_seq_length, tokenizer)
# # # test_features = convert_examples_to_features(test_input_examples, max_seq_length, tokenizer)

# pickle.dump(train_features, open('./gdrive/My Drive/Kaggle_toxic_comments/train_features.p', 'wb'))
# # pickle.dump(test_features, open('./gdrive/My Drive/Kaggle_toxic_comments/test_features.p', 'wb'))

# # ## Load features previously saved
# train_features = pickle.load(open('./gdrive/My Drive/Kaggle_toxic_comments/train_features.p', 'rb'))
# test_features = pickle.load(open('./gdrive/My Drive/Kaggle_toxic_comments/train_features.p', 'rb'))

In [0]:
# utility methods
from collections import namedtuple
NeuralProcessParams = namedtuple('NeuralProcessParams', ['dim_z', 'n_hidden_units_h', 'n_hidden_units_g'])
GaussianParams = namedtuple('GaussianParams', ['mu', 'sigma'])


def batch_mlp(input, inner_layer_dims, output_dim, variable_scope):
  """Apply MLP to the final axis of a 3D tensor (reusing already defined MLPs).
  
  Args:
    input: input tensor of shape [B,n,d_in].
    output_sizes: An iterable containing the output sizes of the MLP as defined 
        in `basic.Linear`.
    variable_scope: String giving the name of the variable scope. If this is set
        to be the same as a previously defined MLP, then the weights are reused.
    
  Returns:
    tensor of shape [B,n,d_out] where d_out=output_sizes[-1]
  """
  # Get the shapes of the input and reshape to parallelise across observations

  output = input

  
  # Pass through MLP
  with tf.variable_scope(variable_scope, reuse=tf.AUTO_REUSE):
    for i, size in enumerate(inner_layer_dims):
      output = tf.nn.relu(
          tf.layers.dense(output, size, name="layer_{}".format(i)))

    # Last layer without a ReLu
    output = tf.layers.dense(output, output_dim, name="layer_{}".format(i + 1))

  return output

In [0]:
# class Embedder(object):
  
#   def __init__(self, BERT_model_hub = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1", trainable=True):
#     self.BERT_model_hub = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
#     self.tokenizer = create_tokenizer_from_hub_module(BERT_model_hub)
#     self.trainable = trainable
    
#   def __call__(self, input_ids, input_mask, segment_ids):
#     embedder = hub.Module(self.BERT_model_hub,trainable=self.trainable)
#     bert_inputs = dict(input_ids=input_ids,
#                        input_mask=input_mask, 
#                        segment_ids=segment_ids)

#     bert_outputs = embedder(inputs=bert_inputs,
#                                signature="tokens", 
#                                as_dict=True)
    
#   # Use "pooled_output" for classification tasks on an entire sentence. Use "sequence_outputs" for token-level output
#     return bert_outputs["pooled_output"]
  

In [0]:
class Decoder(object):
  """The Decoder."""

  def __init__(self, layer_dims, num_classes):
    self.layer_dims = layer_dims
    self.num_classes = num_classes
    
  def __call__(self, input_xs_embedding, z_samples):
  
        # inputs dimensions
    # z_sample has dim [n_draws, dim_z]
    # x_star has dim [N_star, dim_x]
    n_draws = z_samples.get_shape().as_list()[0]
    n_xs = tf.shape(input_xs_embedding)[0]

    # Repeat z samples for each x*
    #z_samples_repeat = tf.expand_dims(z_samples, axis=1)

    #z_samples_repeat = tf.expand_dims(z_samples, axis=1)
    z_samples_repeat = tf.tile(z_samples, [1, n_xs, 1])

    # Repeat x* for each z sample
    x_star_repeat = tf.expand_dims(input_xs_embedding, axis=0)
    x_star_repeat = tf.tile(x_star_repeat, [n_draws, 1, 1])

    # Concatenate x* and z
    inputs = tf.concat([x_star_repeat, z_samples_repeat], axis=2)

    # decoder mlp
    inner_layer_dims = self.layer_dims
    output_dim = self.num_classes *2
    hidden = batch_mlp(inputs, inner_layer_dims, output_dim, "decoder")

    # Get the mean an the variance
    mu, log_sigma = tf.split(hidden, 2, axis= -1)

    # Bound the variance
    sigma_star = 0.1 + 0.9 * tf.nn.softplus(log_sigma)
    mu_star = tf.math.sigmoid(mu)


    return GaussianParams(mu_star, sigma_star)




In [0]:
class Encoder(object):

  def __init__(self, layer_dims, latent_dim):
    self.layer_dims = layer_dims
    self.latent_dim = latent_dim
    
  def __call__(self, xs, ys):
    print(xs)
    print(ys)
    xys = tf.concat([xs, ys], axis=1)


    # encoder mlp
    inner_layer_dims = self.layer_dims[:-1]
    output_dim = self.layer_dims[-1]
    rs = batch_mlp(xys, inner_layer_dims, output_dim, "encoder")
    
    # aggregate rs
    r = self._aggregate_r(rs)
    
    # get mu and sigma
    z_params = self._get_z_params(r)
    
    # distribution
    dist = tfd.MultivariateNormalDiag(loc=z_params.mu,
                                          scale_diag=z_params.sigma)
    return dist
    
  def _aggregate_r(self, context_rs: tf.Tensor) -> tf.Tensor:
    """Aggregate the output of the encoder to a single representation

    Creates an aggregation (mean) operator to combine the encodings of multiple context inputs

    Parameters
    ----------
    context_rs
        Input encodings tensor, shape: (n_samples, dim_r)

    Returns
    -------
        Output tensor of aggregation result
    """
    mean = tf.reduce_mean(context_rs, axis=0)
    r = tf.reshape(mean, [1, -1])
    return r
  
  def _get_z_params(self, context_r: tf.Tensor) -> GaussianParams:
    """Map encoding to mean and covariance of the random variable Z

    Creates a linear dense layer to map encoding to mu_z, and another linear mapping + a softplus activation for Sigma_z

    Parameters
    ----------
    context_r
        Input encoding tensor, shape: (1, dim_r)
    params
        Neural process parameters

    Returns
    -------
        Output tensors of the mappings for mu_z and Sigma_z
    """
    hidden = context_r
    with tf.variable_scope("latent_encoder", reuse=tf.AUTO_REUSE):
      # First apply intermediate relu layer 
      hidden = tf.nn.relu(
          tf.layers.dense(hidden, 
                          (self.layer_dims[-1] + self.latent_dim)/2, 
                          name="penultimate_layer"))
      
      # Then apply further linear layers to output latent mu and log sigma
      mu = tf.layers.dense(hidden, self.latent_dim, name="mean_layer")
      log_sigma = tf.layers.dense(hidden, self.latent_dim, name="std_layer")
      

    # Compute sigma
    sigma = 0.1 + 0.9 * tf.sigmoid(log_sigma)

    return GaussianParams(mu, sigma)

In [0]:
class NLP_NeuralProcess(object):
  
  def __init__(self,
                  score_col, 
                  params = NeuralProcessParams(dim_z=20, 
                                                     n_hidden_units_h=[128, 128, 128], 
                                                     n_hidden_units_g=[128, 128, 128]),
                  num_classes = 6, 
                  num_draws = 2, 
                  lr = 2e-5,
                  batch_size = 32, 
                  num_warmup_steps=100,
                  num_train_steps = 10**3,
                  save_summary_steps = 100,
                  save_checkpoints_steps = 500,
                  keep_checkpoint_max = None,
                  output_dir = "./test_output",
                  context_features = None
                 ):
    
    self.params = params
    self.encoder = Encoder(layer_dims = self.params.n_hidden_units_h, 
                           latent_dim=self.params.dim_z)
    self.decoder = Decoder(layer_dims= self.params.n_hidden_units_g, 
                           num_classes=num_classes)
    self.num_draws = num_draws
    self.num_classes = num_classes
#     self.estimator = None
    #self.embedder = Embedder()
    #####  
    num_labels = len(score_col)
    
    
    # Specify outpit directory and number of checkpoint steps to save
    
    run_config = tf.estimator.RunConfig(model_dir=output_dir,
          save_summary_steps=save_summary_steps, save_checkpoints_steps=save_checkpoints_steps)
    
    model_fn = self.model_fn_builder(num_labels = num_labels, learning_rate=lr,
      num_train_steps=num_train_steps, num_warmup_steps=num_warmup_steps)
    
    if context_features is not None:
      estimator_params = {"batch_size": batch_size, "context_features": context_features}
      self.estimator = tf.estimator.Estimator(model_fn=model_fn,config=run_config,
                                              params=estimator_params)
    
    else: 
      self.estimator = tf.estimator.Estimator(model_fn=model_fn,config=run_config,
            params={"batch_size": batch_size})
    
    
  def create_model(self, 
                   target_input_ids, 
                   target_input_mask, 
                   target_segment_ids, 
                   target_scores=None,
                   context_input_ids = None, 
                   context_input_mask = None, 
                   context_segment_ids= None, 
                   context_scores = None
                   ):
    
    # apply embedder
    embedder = hub.Module(BERT_model_hub,trainable=True)
    
    valid_context = (context_input_ids is not None) & (context_input_mask is not None) & (context_segment_ids is not None) & (context_scores is not None)
    
    # target processing - all scenarios
    target_inputs = dict(input_ids=target_input_ids,
                     input_mask=target_input_mask, 
                     segment_ids=target_segment_ids)
    target_embeddings = embedder(inputs=target_inputs,
                               signature="tokens", 
                               as_dict=True)
    target_xs = target_embeddings["pooled_output"]
    
    
    if valid_context:
      
      # context processing - training
      context_inputs = dict(input_ids=context_input_ids,
                         input_mask=context_input_mask, 
                         segment_ids=context_segment_ids)
      context_embeddings = embedder(inputs=context_inputs,
                                 signature="tokens", 
                                 as_dict=True)
      context_xs = context_embeddings["pooled_output"]
      context_ys = context_scores
      # total x,y 
      x_all = tf.concat([context_xs, target_xs], axis=0)
      
      # get encoding params with context
      context_z_dist = self.encoder(context_xs, context_ys)
      # predictions with context
      posterior_pred = self.decoder(target_xs, context_z_dist.sample(self.num_draws))
        
       # target scores - context training / evaluation
      if target_scores is not None:
        target_ys = target_scores
        y_all = tf.concat([context_ys, target_ys], axis=0)
        all_z_dist = self.encoder(x_all, y_all)
        
        # loss
        loglike = self.loglikelihood(target_ys, posterior_pred)
        KL_loss = self.KLqp_gaussian(all_z_dist.parameters['loc'], 
                                     all_z_dist.parameters['scale_diag'], 
                                     context_z_dist.parameters['loc'], 
                                     context_z_dist.parameters['scale_diag'])
        loss = tf.negative(loglike) + KL_loss
        # context and training / evaluation
        return (loss, posterior_pred, target_ys)
      
      # context prediction
      return  (None, posterior_pred, None)
    
    # no context
    else:
      x_all = target_xs 
      # get internal representation
      mean_zero = tf.constant(np.repeat(0., params.dim_z))
      epsilon_dist = tfd.MultivariateNormalDiag(loc= mean_zero)                            
      epsilon = tf.expand_dims(epsilon_dist.sample(self.num_draws), axis=1)
      epsilon = tf.cast(epsilon, tf.float32)
      prior_predict = self.decoder(x_all, epsilon)
      
      # target scores - no context training / evaluation
      if target_scores is not None:
        target_ys = target_scores
        loglike = self.loglikelihood(target_ys, prior_predict)
        loss = tf.negative(loglike)
        # no context/ training / evaluation
        return  (loss, prior_predict, target_ys)
   
    
      # no context prediction
      return  (None, prior_predict, None)

  
  def loglikelihood(self, y_star: tf.Tensor, dist):
    """Log-likelihood of an output given a predicted """
    p_normal = tfd.MultivariateNormalDiag(loc = dist.mu, scale_diag=dist.sigma)
    loglike = p_normal.log_prob(y_star)
    loglike = tf.reduce_sum(loglike, axis=0)
    loglike = tf.reduce_mean(loglike)
    return loglike
  
  def KLqp_gaussian(self, mu_q: tf.Tensor, sigma_q: tf.Tensor, mu_p: tf.Tensor, sigma_p: tf.Tensor) -> tf.Tensor:
    """Kullback-Leibler divergence between two Gaussian distributions

    Determines KL(q || p) = < log( q / p ) >_q

    Parameters
    ----------
    mu_q
        Mean tensor of distribution q, shape: (1, dim)
    sigma_q
        Variance tensor of distribution q, shape: (1, dim)
    mu_p
        Mean tensor of distribution p, shape: (1, dim)
    sigma_p
        Variance tensor of distribution p, shape: (1, dim)

    Returns
    -------
        KL tensor, shape: (1)
    """
    sigma2_q = tf.square(sigma_q) + 1e-16
    sigma2_p = tf.square(sigma_p) + 1e-16
    temp = sigma2_q / sigma2_p + tf.square(mu_q - mu_p) / sigma2_p - 1.0 + tf.log(sigma2_p / sigma2_q + 1e-16)
    return 0.5 * tf.reduce_sum(temp)
  
  def context_target_split(self, batch_size =32):
    btch_sz = batch_size
    n_context = tf.random_shuffle(tf.range(1,btch_sz))[0]
    
    indices = tf.range(0, btch_sz)
    context_set_indices = tf.gather(tf.random_shuffle(indices),tf.range(n_context))
    target_set_indices = tf.gather(tf.random_shuffle(indices),tf.range(n_context, btch_sz))
    
    return context_set_indices, target_set_indices
    
  def model_fn_builder(self, num_labels, learning_rate, num_train_steps, num_warmup_steps):
      """Returns `model_fn` closure for TPUEstimator."""
      
      
      def model_fn(features, mode, params):  # pylint: disable=unused-argument
          """The `model_fn` for TPUEstimator."""
          
          # run model
          # -------------------------------------------------------------------------------------------
          target_input_ids = None
          target_input_mask = None
          target_segment_ids = None 
          target_scores = None
          
          context_input_ids = None 
          context_input_mask = None 
          context_segment_ids = None
          context_scores = None
          
          # training 
          if mode == tf.estimator.ModeKeys.TRAIN:
            input_ids = features["input_ids"]
            input_mask = features["input_mask"]
            segment_ids = features["segment_ids"]    
            scores = features["scores"]
            
            # context split
            context_set_indices, target_set_indices= self.context_target_split(batch_size =32)
            
            target_input_ids = tf.gather(input_ids,target_set_indices)
            target_input_mask = tf.gather(input_mask,target_set_indices)
            target_segment_ids = tf.gather(segment_ids,target_set_indices) 
            target_scores = tf.gather(scores,target_set_indices)

            context_input_ids = tf.gather(input_ids,context_set_indices) 
            context_input_mask = tf.gather(input_mask,context_set_indices) 
            context_segment_ids = tf.gather(segment_ids,context_set_indices)
            context_scores = tf.gather(scores,context_set_indices)
            
            
          elif mode == tf.estimator.ModeKeys.PREDICT:
            print('Prediction')
            target_input_ids = features["input_ids"]
            target_input_mask = features["input_mask"]
            target_segment_ids = features["segment_ids"]    
            target_scores = features["scores"]
            
            try:
              context_input_ids = features["supplied_context_input_ids"][0]
              context_input_mask = features["supplied_context_input_mask"][0]
              context_segment_ids = features["supplied_context_segment_ids"][0] 
              context_scores = features["supplied_context_scores"][0]
               
            except:
              print("****No context supplied ****")
              context_input_ids = None
              context_input_mask = None
              context_segment_ids = None
              context_scores = None
            
          else:
            print('Evaluation')
            target_input_ids = features["input_ids"]
            target_input_mask = features["input_mask"]
            target_segment_ids = features["segment_ids"]    
            target_scores = features["scores"]

            try:
              context_input_ids = features["supplied_context_input_ids"][0]
              context_input_mask = features["supplied_context_input_mask"][0]
              context_segment_ids = features["supplied_context_segment_ids"][0] 
              context_scores = features["supplied_context_scores"][0]
               
            except:
              print("****No context supplied ****")
              context_input_ids = None
              context_input_mask = None
              context_segment_ids = None
              context_scores = None


          (loss, prediction, true_y) = self.create_model(target_input_ids, 
                                                         target_input_mask, 
                                                         target_segment_ids, 
                                                         target_scores,
                                                         context_input_ids, 
                                                         context_input_mask, 
                                                         context_segment_ids, 
                                                         context_scores)

          train_op = bert.optimization.create_optimizer(loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)
          ystar, _ = tf.nn.moments(prediction.mu,[0])
          variance, _ = tf.nn.moments(tf.math.square(prediction.sigma),[0])
          
          # Calculate evaluation metrics
          eval_metrics = {}
          # AUC
          def metric_fn(pred_scores, real_scores, trait_num):
              auc_value = tf.metrics.auc(real_scores[:,trait_num], pred_scores[:,trait_num])
              accuracy_value = tf.metrics.accuracy(labels = tf.round(real_scores[:,trait_num]), predictions=tf.round(pred_scores[:,trait_num]))
              recall_value = tf.metrics.recall(labels = tf.round(real_scores[:,trait_num]), predictions=tf.round(pred_scores[:,trait_num]))
              precision_value = tf.metrics.precision(labels = tf.round(real_scores[:,trait_num]), predictions=tf.round(pred_scores[:,trait_num]))
              return {"auc"+str(trait_num): auc_value, "accuracy"+str(trait_num): accuracy_value, "recall"+str(trait_num): recall_value, "precision"+str(trait_num):precision_value}

          labels = true_y # need to round them if true labels are not 1 or 0
          eval_metrics_lst = [metric_fn(ystar, labels, trait_num) for trait_num in range(num_labels)]

          for d in eval_metrics_lst:
              tf.summary.scalar(list(d.keys())[0], list(d.values())[0][1])  # make available to tensorboard
              tf.summary.scalar(list(d.keys())[1], list(d.values())[1][1])  # make available to tensorboard
              eval_metrics.update(d)
              
          
          # output from model
          # -------------------------------------------------------------------------------------------

          # training 
          if mode == tf.estimator.ModeKeys.TRAIN:
              return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

          # prediction
          elif mode == tf.estimator.ModeKeys.PREDICT:
              return tf.estimator.EstimatorSpec(mode=mode, predictions={'prediction_mean': ystar, 'prediction_var': variance})

          # evaluation
          else:
            return tf.estimator.EstimatorSpec(mode=mode,loss=loss, eval_metric_ops=eval_metrics)

      return model_fn


  def prepare_examples(self, df, score_column, text_col_name, supplied_context_df=None):
      num_labels = len(score_column)
      input_examples = create_examples(df, score_column, text_col_name)
      input_features = convert_examples_to_features(input_examples, max_seq_length, tokenizer)
      
      if supplied_context_df is not None:
        supplied_context_examples = create_examples(supplied_context_df, score_column, text_col_name)
        supplied_context_features = convert_examples_to_features(supplied_context_examples, max_seq_length, tokenizer)
        input_fn = input_fn_builder(
          features=input_features, seq_length=max_seq_length, 
          num_labels = num_labels, is_training=True, drop_remainder=False,
          supplied_context_features = supplied_context_features)
      
      else:
        input_fn = input_fn_builder(
          features=input_features, seq_length=max_seq_length, 
          num_labels = num_labels, is_training=True, drop_remainder=False)
        
      return input_fn
  
  def predict(self, 
            df, 
            score_col, 
            text_col,
            supplied_context_df=None
             ):
    
    pred_input_fn = self.prepare_examples(df, score_col, text_col, supplied_context_df)
    preds = self.estimator.predict(input_fn=pred_input_fn)
    
    return preds
  
  def evaluate(self, 
               eval_steps,
               df, 
               score_col, 
               text_col,
               supplied_context_df=None):
    
    eval_input_fn = self.prepare_examples(df, score_col, text_col, supplied_context_df)
    
    result = self.estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
    return result
  
  def train(self,num_train_steps,
            df_train, 
            score_col, 
            text_col,
           ):

    # Create an input function for training. drop_remainder = True for using TPUs.
    train_input_fn = self.prepare_examples(df_train, score_col, text_col)

    print('Beginning Training!')
    current_time = datetime.now()
    self.estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
    print("Training took time ", datetime.now() - current_time)
    
  
  def train_and_evaluate(self,
                         df_train,
                         df_eval,
                         score_col, 
                         text_col,
                         num_train_steps,
                         eval_steps,
                         supplied_context_df=None):
    
    train_input_fn = self.prepare_examples(df_train, score_col, text_col)
    eval_input_fn = self.prepare_examples(df_eval, score_col, text_col, supplied_context_df)
    
    train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=num_train_steps)
    eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn, steps = eval_steps)
    
    result = tf.estimator.train_and_evaluate(self.estimator, train_spec, eval_spec)
    return result
    

In [0]:

# tf.reset_default_graph()

params = NeuralProcessParams(dim_z=1000, n_hidden_units_h=[512, 256, 128], n_hidden_units_g=[512, 256, 128])
num_train_steps = 10*(10**3)

neural_process = NLP_NeuralProcess(score_col=score_column, params = params, num_draws = 25, 
                                   num_train_steps=num_train_steps,
                                   num_classes = len(score_column),
                                   output_dir = OUTPUT_DIR,
                                  context_features = None)

I0808 11:05:58.068946 139698332669824 estimator.py:209] Using config: {'_model_dir': 'gs://rail_jigsaw_modeling/unhealthy/models/batch32', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0d60e29e48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
# # df_test = test_df
# batch_size = 32
# eval_steps = int(len(df_val) / batch_size)

# neural_process.train_and_evaluate(df_train=df_train,
#                                   df_eval = df_val,
#                                   num_train_steps=num_train_steps,
#                                   eval_steps = eval_steps,
#                                   score_col= score_column, 
#                                   text_col=text_col_name,)

In [0]:
random_context = df_train.sample(20, random_state=2)

In [0]:
batch_size = 32
eval_steps = int(len(df_test) / batch_size)

neural_process.evaluate(eval_steps,
                       df_test, 
                       score_col= score_column, 
                       text_col=text_col_name, supplied_context_df = random_context)

I0808 11:39:29.046445 139698332669824 estimator.py:1145] Calling model_fn.


Evaluation


I0808 11:39:32.515992 139698332669824 saver.py:1499] Saver not created because there are no variables in the graph to restore
I0808 11:39:33.721351 139698332669824 saver.py:1499] Saver not created because there are no variables in the graph to restore


Tensor("module_apply_tokens_1/bert/pooler/dense/Tanh:0", shape=(20, 768), dtype=float32)
Tensor("strided_slice_3:0", shape=(20, 6), dtype=float32)
Tensor("concat:0", shape=(?, 768), dtype=float32)
Tensor("concat_3:0", shape=(?, 6), dtype=float32)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
I0808 11:39:47.282434 139698332669824 estimator.py:1147] Done calling model_fn.
I0808 11:39:47.302659 139698332669824 evaluation.py:255] Starting evaluation at 2019-08-08T11:39:47Z
I0808 11:39:48.784996 139698332669824 monitored_session.py:240] Graph was finalized.
I0808 11:39:49.020799 139698332669824 saver.py:1280] Restoring parameters from gs://rail_jigsaw_modeling/unhealthy/models/batch32/model.ckpt-30000
I0808 11:41:16.657028 139698332669824 session_manager.py:500] Running local_init_op.
I0808 11:41:17.081788 139698332669824 session_manager.py:502] Done running local_init_op.
I0808 11:41:27.194047 139698332669824 evaluation.py:167] Evaluation [14/140]
I0808 11:41:34.401983 139698332669824 evaluation.py:167

{'accuracy0': 0.9375,
 'accuracy1': 0.91450894,
 'accuracy2': 0.9558036,
 'accuracy3': 0.97098213,
 'accuracy4': 0.96875,
 'accuracy5': 0.92901784,
 'auc0': 0.57798177,
 'auc1': 0.60003,
 'auc2': 0.5898204,
 'auc3': 0.55191123,
 'auc4': 0.5430317,
 'auc5': 0.5617589,
 'global_step': 30000,
 'loss': 132.25116,
 'precision0': 0.3164557,
 'precision1': 0.23880596,
 'precision2': 0.2195122,
 'precision3': 0.2037037,
 'precision4': 0.5,
 'precision5': 0.14285715,
 'recall0': 0.0996016,
 'recall1': 0.17266187,
 'recall2': 0.05142857,
 'recall3': 0.1122449,
 'recall4': 0.05,
 'recall5': 0.078947365}

In [0]:
df_test2 = df_test[df_test._trusted_judgments<6]

In [0]:
preds = neural_process.predict(
                       df_test2, 
                       score_col= score_column, 
                       text_col=text_col_name,supplied_context_df = random_context)

In [0]:

test_scores = df_test2[score_column]

In [0]:
uncertainty_per_score = pd.DataFrame(columns=['presence','accuracy', 'variance', 'num_judgments'])
uncertainty_per_row = pd.DataFrame(columns=['presence','accuracy', 'variance', 'num_judgments'])
pred_means = pd.DataFrame(columns = score_column)
pred_variances = pd.DataFrame(columns = score_column)

for i, pred in enumerate(preds):
  if i<len(test_scores):
    scores = np.array(test_scores.iloc[i])
    #     print(scores)
    pred_means = pred_means.append({score_column[k]:pred['prediction_mean'][k] for k in range(len(score_column))}, ignore_index=True)
    #     print(mean_prediction)
    variance = pred['prediction_var']
    pred_variances = pred_variances.append({score_column[k]:variance[k] for k in range(len(score_column))}, ignore_index=True)  
    mean_prediction = np.round(pred['prediction_mean'])
    acc = scores==mean_prediction
    row_acc = int( sum(scores==mean_prediction)==len(scores) )
    presence = int(sum(scores)>0)
    uncertainty_per_row = uncertainty_per_row.append({'presence': presence,'accuracy': row_acc, 'variance': np.mean(variance), 'num_judgments':df_test.iloc[i,-1]}, ignore_index=True)
    for j in range(len(scores)):
      uncertainty_per_score = uncertainty_per_score.append({'presence':scores[j], 'accuracy': int(acc[j]), 'variance': variance[j], 'num_judgments':int(df_test.iloc[i,-1])}, ignore_index=True)
  else:
    break

I0808 11:20:07.354612 139698332669824 estimator.py:1145] Calling model_fn.


Prediction


I0808 11:20:11.079793 139698332669824 saver.py:1499] Saver not created because there are no variables in the graph to restore
I0808 11:20:11.952297 139698332669824 saver.py:1499] Saver not created because there are no variables in the graph to restore


Tensor("module_apply_tokens_1/bert/pooler/dense/Tanh:0", shape=(300, 768), dtype=float32)
Tensor("strided_slice_3:0", shape=(300, 6), dtype=float32)
Tensor("concat:0", shape=(?, 768), dtype=float32)
Tensor("concat_3:0", shape=(?, 6), dtype=float32)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
I0808 11:20:25.788328 139698332669824 estimator.py:1147] Done calling model_fn.
I0808 11:20:27.293401 139698332669824 monitored_session.py:240] Graph was finalized.
I0808 11:20:27.525445 139698332669824 saver.py:1280] Restoring parameters from gs://rail_jigsaw_modeling/unhealthy/models/batch32/model.ckpt-30000
I0808 11:21:16.229130 139698332669824 session_manager.py:500] Running local_init_op.
I0808 11:21:16.643101 139698332669824 session_manager.py:502] Done running local_init_op.


In [0]:
uncertainty_per_score.corr()

,presence,accuracy,variance,num_judgments
presence,1.000000,-0.805235,-0.007815,0.000882
accuracy,-0.805235,1.000000,-0.213548,0.003375
variance,-0.007815,-0.213548,1.000000,-0.003348
num_judgments,0.000882,0.003375,-0.003348,1.000000


In [0]:
uncertainty_per_row.corr()

,presence,accuracy,variance,num_judgments
presence,1.000000,-0.752693,-0.010856,-0.006546
accuracy,-0.752693,1.000000,-0.287406,0.014728
variance,-0.010856,-0.287406,1.000000,-0.005299
num_judgments,-0.006546,0.014728,-0.005299,1.000000


In [0]:
print("Average variance for accurate/correct predictions:")
print(uncertainty_per_score[uncertainty_per_score.accuracy==1]['variance'].mean())
print("Average variance for inaccurate/false predictions:")
print(uncertainty_per_score[uncertainty_per_score.accuracy==0]['variance'].mean())
print("Ratio of average variance incorrect::correct")
print(uncertainty_per_score[uncertainty_per_score.accuracy==0]['variance'].mean()/uncertainty_per_score[uncertainty_per_score.accuracy==1]['variance'].mean())

Average variance for accurate/correct predictions:
0.01892358005760754
Average variance for inaccurate/false predictions:
0.1041500688000671
Ratio of average variance incorrect::correct
5.503719089253269


Check for correlation with presence/absence of attribtue with variance:

In [0]:
print("Average variance for predictions when attribute is present:")
print(uncertainty_per_score[uncertainty_per_score.presence==1]['variance'].mean())
print("Average variance for predictions when attribute is absent:")
print(uncertainty_per_score[uncertainty_per_score.presence==0]['variance'].mean())
print("Ratio of average variance present::absent")
print(uncertainty_per_score[uncertainty_per_score.presence==1]['variance'].mean()/uncertainty_per_score[uncertainty_per_score.presence==0]['variance'].mean())

Average variance for predictions when attribute is present:
0.02882015669502065
Average variance for predictions when attribute is absent:
0.023344012376548595
Ratio of average variance present::absent
1.2345845362887742


In [0]:
uncertainty_per_row = pd.DataFrame(columns=['presence','accuracy', 'variance', 'num_judgments'])
pred_means = pd.DataFrame(columns = score_column)
pred_variances = pd.DataFrame(columns = score_column)

for i, pred in enumerate(preds):
  if i<len(test_scores):
    scores = np.array(test_scores.iloc[i])
    pred_means = pred_means.append({score_column[k]:pred['prediction_mean'][k] for k in range(len(score_column))}, ignore_index=True)
    variance = pred['prediction_var']
    pred_variances = pred_variances.append({score_column[k]:variance[k] for k in range(len(score_column))}, ignore_index=True)  
    mean_prediction = np.round(pred['prediction_mean'])
    row_acc = int( sum(scores==mean_prediction)==len(scores) )
    presence = int(sum(scores)>0)
    uncertainty_per_row = uncertaintyper_row.append({'presence': presence,'accuracy': acc, 'variance': np.mean(variance), 'num_judgments':df_test.iloc[i,-1]}, ignore_index=True)
  else:
    break

In [0]:
def total_accuaracy(pred_means, test_scores):
  preds = np.round(pred_means)
  comparison = preds == test_scores
  correct_guesses = np.reshape(comparison.values,-1)
  return np.sum(correct_guesses) / len(correct_guesses)

In [0]:
def class_accuaracy(pred_means, test_scores):
  preds = np.round(pred_means)
  comparison = preds == test_scores
  correct_guesses = comparison.values
  return correct_guesses.sum(axis=0)/ len(correct_guesses)


In [0]:
print(total_accuaracy(pred_means, test_scores.reset_index(drop=True)))
print(class_accuaracy(pred_means, test_scores.reset_index(drop=True)))

0.9060351812645391
[0.79283306 0.95742178 0.8519564  0.99568729 0.87359837 0.96471418]


In [0]:
def subset_accuracy(att, att_num):
  print("accuracy on "+att+ " comments:")
  print(class_accuaracy(pred_means.iloc[np.where(test_scores[att] == 1)].reset_index(drop=True),
                        test_scores.iloc[np.where(test_scores[att] == 1)].reset_index(drop=True))[att_num])

  print("proportion of "+att+ " comments:")
  print(len(test_scores.iloc[np.where(test_scores[att] == 1)].reset_index(drop=True))/len(test_scores))
  print()

for i, att in enumerate(score_column):
  subset_accuracy(att, i)

accuracy on toxic comments:
0.11676646706586827
proportion of toxic comments:
0.1309495804908649

accuracy on severe_toxic comments:
0.030864197530864196
proportion of severe_toxic comments:
0.012702893436838392

accuracy on obscene comments:
0.09008097165991903
proportion of obscene comments:
0.07747196738022426

accuracy on threat comments:
0.0
proportion of threat comments:
0.004312710734729083

accuracy on insult comments:
0.06285072951739619
proportion of insult comments:
0.06986591390261115

accuracy on identity_hate comments:
0.013333333333333334
proportion of identity_hate comments:
0.011761938367442954



In [0]:
len(test_scores[test_scores.severe_toxic==1])

229

In [0]:
at_least_one = (test_scores.toxic==1) | (test_scores.severe_toxic==1) | \
               (test_scores.obscene==1) | (test_scores.threat==1) | \
               (test_scores.insult==1) | (test_scores.identity_hate==1)

In [0]:
print(uncertainty[uncertainty.accuracy==True]['variance'].mean())
print(uncertainty[uncertainty.accuracy==False]['variance'].mean())

37299

In [0]:
try: 
  print("this")
  True/"this"
except:
  print("didnt work")

this
didnt work


In [0]:
unhealthy_df = df_train[(df_train.antagonize == 1) | (df_train.condescending == 1) |(df_train.dismissive == 1) | (df_train.generalisation == 1) |
                 (df_train.hostile == 1) | (df_train.sarcastic == 1)]
healthy_df = df_train[(df_train.antagonize == 0) & (df_train.condescending == 0) & (df_train.dismissive == 0) & (df_train.generalisation == 0) &
                 (df_train.hostile == 0) & (df_train.sarcastic == 0)]

In [0]:
context_df = pd.concat([unhealthy_df.sample(10), healthy_df.sample(40)])

In [0]:
batch_size = 32
eval_steps = int(len(df_test) / batch_size)

neural_process.evaluate(eval_steps,
                       df_test, 
                       score_col= score_column, 
                       text_col=text_col_name, supplied_context_df = context_df)

I0706 16:53:42.960248 140042348476288 estimator.py:1145] Calling model_fn.


Evaluation


I0706 16:53:46.208950 140042348476288 saver.py:1499] Saver not created because there are no variables in the graph to restore
I0706 16:53:46.811548 140042348476288 saver.py:1499] Saver not created because there are no variables in the graph to restore


Tensor("module_apply_tokens_1/bert/pooler/dense/Tanh:0", shape=(50, 768), dtype=float32)
Tensor("strided_slice_3:0", shape=(50, 6), dtype=float32)
Tensor("concat:0", shape=(?, 768), dtype=float32)
Tensor("concat_3:0", shape=(?, 6), dtype=float32)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
I0706 16:53:59.571602 140042348476288 estimator.py:1147] Done calling model_fn.
I0706 16:53:59.595152 140042348476288 evaluation.py:255] Starting evaluation at 2019-07-06T16:53:59Z
I0706 16:54:01.813944 140042348476288 monitored_session.py:240] Graph was finalized.
I0706 16:54:02.037882 140042348476288 saver.py:1280] Restoring parameters from gs://rail_jigsaw_modeling/unhealthy/models/batch32/model.ckpt-30000
I0706 16:54:44.575266 140042348476288 session_manager.py:500] Running local_init_op.
I0706 16:54:45.075836 140042348476288 session_manager.py:502] Done running local_init_op.
I0706 16:54:59.687331 140042348476288 evaluation.py:167] Evaluation [14/140]
I0706 16:55:10.956083 140042348476288 evaluation.py:167

{'accuracy0': 0.9375,
 'accuracy1': 0.9140625,
 'accuracy2': 0.9558036,
 'accuracy3': 0.9707589,
 'accuracy4': 0.96875,
 'accuracy5': 0.93013394,
 'auc0': 0.5799889,
 'auc1': 0.598453,
 'auc2': 0.58911043,
 'auc3': 0.5513847,
 'auc4': 0.5432443,
 'auc5': 0.5626207,
 'global_step': 30000,
 'loss': 133.39682,
 'precision0': 0.3164557,
 'precision1': 0.24257426,
 'precision2': 0.225,
 'precision3': 0.2037037,
 'precision4': 0.46666667,
 'precision5': 0.15322581,
 'recall0': 0.0996016,
 'recall1': 0.17437722,
 'recall2': 0.051136363,
 'recall3': 0.11111111,
 'recall4': 0.05035971,
 'recall5': 0.08370044}